In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
def get_html(url):
    r = requests.get(url)
    return r.text
def get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    pages = soup.find('div', class_='search-result-paging').find_all('a')
    total_pages = pages[-2].get('href').split('/')[-2]
    return int(total_pages)
def get_page_data(html):
    soup = BeautifulSoup(html, 'lxml')
    ads = soup.find('div', class_='main-wrap__content').find_all('div', class_='company-item')
    spisok = []
    for ad in ads:
        
        try:
            title = ad.find('div', class_='company-item__title').find('a').text.strip()
        except:
            title = ''
        try:
            url = 'https://www.rusprofile.ru'+str(ad.find('div', class_='company-item__title').find('a').get('href'))
        except:
            url = ''
        try:
            address = ad.find('address').text.strip()
        except:
            address = ''
        ad_subs = ad.find_all('div', class_='company-item-info')
        try:
            manager = ad_subs[0].find('dt').text
        except:
            manager = ''
        try:
            manager_name = ad_subs[0].find('dd').text
        except:
            manager_name = ''
        try:
            INN = ad_subs[1].find('dt').text
        except:
            INN = ''
        try:
            INN_value = ad_subs[1].find('dd').text
        except:
            INN_value = ''
        try:
            dealing = ad_subs[2].find('dt').text
        except:
            dealing = ''
        try:
            okved = ad_subs[2].find('dd').text
        except:
            okved = ''
        

        data = {'Название компании': title, 'URL-адрес': url,
               'Адрес': address, 'Руководитель': manager,
                'ФИО руководителя': manager_name,
                'Идентификатор': INN, 'ИНН': INN_value,
                'Вид деятельности': dealing, 'ОКВЭД': okved}
        spisok.append(data)
        
        
    return spisok

In [94]:
def main():
    url = 'https://www.rusprofile.ru/codes/683210/moskva'
    base_url = 'https://www.rusprofile.ru/codes/683210/moskva/'
    total_pages = get_total_pages(get_html(url))
    spisok = []
    for i in range(1, 3):
        url_gen = base_url + str(i)
        html = get_html(url_gen)
        spisok.extend(get_page_data(html))
    return spisok

In [95]:
okved_683210_moskva = pd.DataFrame(main())
okved_683210_moskva.to_excel('okved_683210_moskva.xlsx')

In [2]:
okved_683210_moskva = pd.read_excel('okved_683210_moskva.xlsx')

In [32]:
urls = okved_683210_moskva['URL-адрес']

In [35]:
spisok = []
for url in urls:
    html = get_html(url)
    soup = BeautifulSoup(html, 'lxml')
    try:
        ads = soup.find_all('div', class_='change-item')
        for ad in ads:
            date_change = ad.find('div', class_='change-item__date').text
            text_change = ad.find('div', class_='change-item__text').text.strip()
            data = {'URL-адрес': url, 'Дата изменений': date_change, 'Текст изменений': text_change}
            spisok.append(data)
    except:
        pass
changes_in_okved_683210_moskva = pd.DataFrame(spisok)
changes_in_okved_683210_moskva.to_excel('changes_in_okved_683210_moskva.xlsx')